In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from sqlalchemy import create_engine

In [2]:
pip install sqlalchemy --upgrade


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --- ------------------------------------ 0.2/2.1 MB 5.0 MB/s eta 0:00:01
   ------- -------------------------------- 0.4/2.1 MB 6.1 MB/s eta 0:00:01
   ----------- ---------------------------- 0.6/2.1 MB 5.4 MB/s eta 0:00:01
   ------------ --------------------------- 0.7/2.1 MB 4.2 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/2.1 MB 3.8 MB/s eta 0:00:01
   ----------------------- ---------------- 1.2/2.1 MB 4.5 MB/s eta 0:00:01
   ---------------------------- ----------- 1.5/2.1 MB 4.5 MB/s eta 0:00:01
   --------------------------------- ------ 1.7/2.1 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 4.7 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.39
    Uninstalling SQLAlchemy-1.4.39:
      Successfully uninstalled SQLAlchemy-1.4.39